In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pyeunju","key":"50a8c9df56c4f9dbb1221d49060c9b8c"}'}

In [2]:
ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c 2022-samsung-ds-competition-co-gas-data

 95% 180M/189M [00:04<00:00, 66.7MB/s]
100% 189M/189M [00:04<00:00, 48.9MB/s]


In [5]:
!ls

2022-samsung-ds-competition-co-gas-data.zip  kaggle.json  sample_data


In [6]:
!unzip 2022-samsung-ds-competition-co-gas-data.zip

Archive:  2022-samsung-ds-competition-co-gas-data.zip
  inflating: sample_solution.csv     
  inflating: test_data.csv           
  inflating: train_data.csv          


In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, 
                              AdaBoostRegressor)
from sklearn.linear_model import LinearRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
training_data = pd.read_csv('train_data.csv')
test_input_data = pd.read_csv('test_data.csv')

In [9]:
# # Signal shift off
# train_x = np.array(training_data.values[:,3:], dtype=np.float64)
# train_y = np.array(training_data.values[:,2], dtype=np.float64)
# # test_x = np.array(test_input_data.values[:,18:], dtype=np.float64)
# test_x = np.array(test_input_data.values[:,2:], dtype=np.float64)

In [10]:
# Signal shift on
cols = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16']
for s in cols:
    training_data['shf_{}'.format(s)] = training_data['{}'.format(s)].shift(-2000)
    test_input_data['shf_{}'.format(s)] = test_input_data['{}'.format(s)].shift(-2000)
training_data = training_data.dropna()
test_input_data = test_input_data.fillna(0)

train_x = np.array(training_data.values[:,19:], dtype=np.float64)
train_y = np.array(training_data.values[:,2], dtype=np.float64)
test_x = np.array(test_input_data.values[:,18:], dtype=np.float64)
# test_x = np.array(test_input_data.values[:,2:], dtype=np.float64)

In [11]:
# def arr_agg_by_sliding_window(arr: np.ndarray, _agg_func: str, window_size):
#     """
#     MIN    | 'min'
#     MAX    | 'max'
#     MEDIAN | 'med'
#     MEAN   | 'avg'
#     """
#     from scipy.ndimage.filters import (minimum_filter, maximum_filter, median_filter)
#     # from scipy.ndimage import uniform_filter # 

#     if _agg_func in ('min', 'max', 'med'):
#         if _agg_func == 'min': agg_func = minimum_filter
#         if _agg_func == 'max': agg_func = maximum_filter
#         if _agg_func == 'med': agg_func = median_filter
#         dim_a = len(arr.shape)
#         N, M = 1, window_size
#         if dim_a == 1:
#             P, Q = len(arr), 1
#             maxs = agg_func(arr.reshape((-1, 1)), size=(M, N))
#         elif dim_a == 2:
#             P,Q = arr.shape
#             maxs = agg_func(arr, size=(M, N))
#         ret = maxs[M//2:(M//2)+P-M+1, N//2:(N//2)+Q-N+1]
        
#         if dim_a == 1:
#             return ret.reshape((-1))
#         if dim_a == 2:
#             ret = np.pad(ret, ((0,arr.shape[0]-ret.shape[0]),(0,0)), 'constant', constant_values=0)
#             return ret
    
#     else: # moving average
#         import bottleneck as bn
#         ret = bn.move_mean(arr, window = window_size, axis=0)[window_size-1:]
#         ret = np.pad(ret, ((0,arr.shape[0]-ret.shape[0]),(0,0)), 'constant', constant_values=0)
#         return ret

# #mooving average

# interval = 10 #10
# train_x_avg = arr_agg_by_sliding_window(train_x, 'avg', interval)
# train_x = np.concatenate([train_x, train_x_avg],1)

# test_x_avg = arr_agg_by_sliding_window(test_x, 'avg', interval)
# test_x = np.concatenate([test_x, test_x_avg],1)

In [12]:
# filtering on
from scipy.signal import butter, filtfilt, square
import matplotlib.pyplot as plt

# b, a = butter(4, 0.0055/(100/2), btype='low', analog=False)  # fs = 100 Hz, fc = 0.005 Hz
b, a = butter(4, 1/(100/2), btype='low', analog=False)  # fs = 100 Hz, fc = 1 Hz
filt_train_data = np.array([filtfilt(b, a, train_x[:,i]) for i in range(len(train_x[0]))]).T
train_x = filt_train_data
filt_test_data = np.array([filtfilt(b, a, test_x[:,i]) for i in range(len(test_x[0]))]).T
test_x = filt_test_data

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [14]:
split_p = int(len(train_x)*0.2)

train1_x = train_x[:split_p*1,:] 
train2_x = train_x[split_p*2:,:] 
train_x_t = np.concatenate([train1_x,train2_x])
train_x_v = train_x[split_p*1:split_p*2,:]

train1_y = train_y[:split_p*1] 
train2_y = train_y[split_p*2:] 
train_y_t = np.concatenate([train1_y,train2_y],)
train_y_v = train_y[split_p*1:split_p*2]

In [15]:
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(data[i:i+window_size])
        label_list.append(label[i:i+window_size])
    return np.array(feature_list), np.array(label_list)

def make_dataset2(data, window_size=20):
    feature_list = []
    for i in range(len(data) - window_size):
        feature_list.append(data[i:i+window_size])
    return np.array(feature_list)

In [16]:
train_x_t_seq, train_y_t_seq = make_dataset(train_x_t, train_y_t, 10)
train_x_v_seq, train_y_v_seq = make_dataset(train_x_v, train_y_v, 10)
train_x_seq, train_y_seq = make_dataset(train_x, train_y, 10)

In [17]:
print(train_x_t_seq.shape)
print(train_y_t_seq.shape)
print(train_x_v_seq.shape)
print(train_y_v_seq.shape)
print(train_x_seq.shape)
print(train_y_seq.shape)

(2691677, 10, 16)
(2691677, 10)
(672911, 10, 16)
(672911, 10)
(3364598, 10, 16)
(3364598, 10)


In [18]:
test_x_seq = make_dataset2(test_x, 10)
test_x_seq.shape

(841643, 10, 16)

In [19]:
from keras.models import Sequential
from keras.layers import Dense , Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Dropout

In [20]:
model_cnn =Sequential()
# model_cnn.add(Conv1D(input_shape = (train_feature.shape[1], train_feature.shape[2]), 
#                         filters=64,kernel_size=3,activation='relu'))
model_cnn.add(Conv1D(input_shape = (train_x_t_seq.shape[1], train_x_t_seq.shape[2]), 
                        filters=64,kernel_size=3,activation='relu'))
#model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling1D(pool_size = 2))
model_cnn.add(Dropout(0.25))


model_cnn.add(Conv1D(filters=32,kernel_size=3,activation='relu'))
#model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling1D(pool_size = 2))
model_cnn.add(Dropout(0.25))

#model_cnn.add(Conv1D(filters=16,kernel_size=3,activation='relu'))
#model_cnn.add(MaxPooling1D(pool_size = 2))
#model_cnn.add(Dropout(0.25))

model_cnn.add(Flatten())
model_cnn.add(Dense(1000))
model_cnn.add(Dropout(0.25))
model_cnn.add(Activation('relu'))

model_cnn.add(Dense(1))
model_cnn.add(Activation('linear'))
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 8, 64)             3136      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4, 64)            0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 4, 64)             0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2, 32)             6176      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 1, 32)            0         
 1D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 1, 32)             0

In [ ]:
import os
model_cnn.compile(loss='mean_absolute_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=20)

model_path = '/content/'
filename = os.path.join(model_path, 'tmp_checkpoint_cnn_all_shft.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

# history = model_cnn.fit(train_x_seq, train_y_seq,
history = model_cnn.fit(train_x_t_seq, train_y_t_seq,
                                    epochs=1, 
                                    # epochs=200, 
                                    batch_size=16,
                                    validation_data=(train_x_v_seq, train_y_v_seq), 
                                    callbacks=[early_stop, checkpoint])

101672/168230 [=================>............] - ETA: 4:06 - loss: 13.4920

In [ ]:
model_cnn.load_weights(filename)
pred = model_cnn.predict(test_x_seq)
print(pred.shape)
print(pred.min(), pred.max())

In [ ]:
# def post_process_pred_ver2(y_pred, y_class_val_list):
#     """
#     use 'train_y' for 'y_class_val_list'
#     """
#     y_cls = np.sort(np.unique(y_class_val_list))
#     y_cls_mid = [(y_cls[i] + y_cls[i + 1])/2 for i in range(len(y_cls) - 1)]
#     y_pred_mapped = y_cls[np.searchsorted(y_cls_mid, y_pred)]
#     return y_pred_mapped

# pred2 = post_process_pred_ver2(pred, np.unique(train_y))


In [ ]:
# sample_submission = pd.read_csv('sample_solution.csv')
# pred = np.pad(pred, ((0,sample_submission.shape[0]-pred.shape[0]),(0,0)), 'constant', constant_values=0)

In [ ]:
# sample_submission['CO'] = pred
# sample_submission.to_csv('predicted_output.csv', index=False) 
# !kaggle competitions submit -c 2022-samsung-ds-competition-co-gas-data -f predicted_output.csv -m "Test"

In [ ]:
# print(pred3.min(), pred3.max())

In [ ]:
# len(np.unique(pred3))

In [ ]:
# np.unique(pred3)

In [ ]:
class Y_PRED_CLASSIFIER:
    def __init__(self, y_pred, y_real):
        self.y_pred_clf = self.create_y_pred_clf(y_pred, y_real)

    def create_y_pred_clf(self, y_pred, y_real):
        """
        """
        from sklearn.naive_bayes import GaussianNB
        y_pred_clf = GaussianNB().fit(y_pred.reshape(-1, 1), y_real.astype(str))
        return y_pred_clf

    def post_process_pred_ver4(self, y_pred):
        """
        y_pred_clf = create_y_pred_clf(y_pred, y_real):
        """
        y_pred_mapped = self.y_pred_clf.predict(y_pred.reshape(-1, 1)).astype(float)
        return y_pred_mapped
      

In [ ]:
pp = model_cnn.predict(train_x_seq)
pp2 = np.pad(pp, ((0,train_y.shape[0]-pp.shape[0]),(0,0)), 'constant', constant_values=0)
pp3 = np.squeeze(pp2)
y_pred_clf = Y_PRED_CLASSIFIER(pp3, train_y) ## create y pred clf 
y_pred = y_pred_clf.post_process_pred_ver4(pred)  # using y pred clf

In [ ]:
sample_submission = pd.read_csv('sample_solution.csv')

In [ ]:
y_pred2 = np.pad(y_pred, ((0,sample_submission.shape[0]-y_pred.shape[0])), 'constant', constant_values=0)

In [ ]:
sample_submission['CO'] = y_pred2  #pred3
sample_submission.to_csv('predicted_output.csv', index=False) 
!kaggle competitions submit -c 2022-samsung-ds-competition-co-gas-data -f predicted_output.csv -m "Test"

In [ ]:
import matplotlib.pyplot as plt
plt.hist(y_pred2)
plt.show()

In [ ]:
plt.hist(pred)
plt.show()

In [ ]:
# multioutputregressor = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
#               max_depth = 10, alpha = 10, n_estimators = 100, eval_metric='mae')
# multioutputregressor.fit(x_train, y_train
#                       ,eval_set=[(x_test, y_test)],
#                        early_stopping_rounds = 10, eval_metric='mae')

In [ ]:
np.unique(train_y)

In [ ]:
#######################여기까지

In [ ]:
import datetime
print(datetime.datetime.now())